# Model Free Control

> A notebook that helps us to discover Reinforcement Learning
- toc: true
- branch: master
- badges: true
- comments: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2
- image: https://www.mdpi.com/symmetry/symmetry-12-01685/article_deploy/html/images/symmetry-12-01685-g002.png
- description: Fourth in a series on understanding Reinforcement Learning.

# Objectives

Previously, we have learned qbout [Model Free Prediction](https://dnlam.github.io/fastblog/2020/06/21/Model_Free_Prediction_Control.html) which enables us to estimate the value function of unknown MDP using sampling (Monte Carlo and Temporal Different Learning Algorithms). 

In this notebook, we will talk about model free control which we will try to optimize these value functions.

# Monte Carlo Control